In [6]:
import numpy as np
import matplotlib.pyplot as plt
import ellc
from scipy.optimize import minimize

## Performing an ELLC fit to the TESS lightcurve data


In [3]:
# TODO: extract the timestamps, fluxes, and flux errors for your TESS object 
# TESS lightcurves are located in /scratch/submit/tess/data/tesscurl_sector_{sector_number}_lc
t = []
y = []
dy = []

In [4]:
def array_ellc_model(params, t, period):
    A = params[0]
    t_zero = params[1]
    radius_1 = params[2]
    radius_2 = params[3]
    incl = params[4]
    sbratio = params[5]
    heat_1 = params[6]
    heat_2 = params[7]
    q = params[8]
    # The ellc model throws errors when a star's radius overflows it's Roche limit
    # Hint: Look at https://en.wikipedia.org/wiki/Roche_lobe to figure whether or not
    # you should attempt to use ellc in the first place
    try:
        model = A * ellc.lc(
            t,
            t_zero=t_zero,
            q=q,
            radius_1=radius_1,
            radius_2=radius_2,
            incl=incl,
            sbratio=sbratio,
            shape_1="roche",
            shape_2="roche",
            period=period,
            heat_1=heat_1,
            heat_2=heat_2,
        )
    except:
        print("Failed with parameters: ", params)
        return t * 10**99

    return model


def fcn2min(params, t, y, dy, period):
    model = array_ellc_model(params, t, period)
    residuals = (y - model) / dy
    return np.sum(residuals**2)

In [5]:
# we can use scipy.optimize.minimize to find the set of parameters that minimizes the objective function
# we provide the function with our initial guess of the parameters and give it bounds to define the search space
guess = [
    1,  # scaling factor
    5.72234658e-03,  # t_zero
    0.05,  # radius_1
    3.91802709e-01,  # radius_2
    8.08703949e01,  # incl
    0.2,  # sbratio
    0.8,  # heat_1
    0.5,  # heat_2
    0.7,  # q
]
bounds = [
    (0.9, 1.2),  # A = scaling factor for light curve
    (0.0057, 0.0058),  # t_zero
    (0.01, 0.18),  # radius_1
    (0.3, 0.41),  # radius_2
    (80, 90),  # incl
    (0, 1),  # sbratio
    (0, 10),  # heat_1
    (0, 10),  # heat_2
    (0.1, 1.0),  # q
]
# TODO: if we knew the period beforehand, this would make the curve-fitting a lot faster and easier. How can we obtain the period for the light curve?
period = 0

res = minimize(
    fcn2min, x0=guess, args=(t, y, dy, period), method="powell", bounds=bounds
)

Failed with parameters:  [1.00000000e+00 5.72234658e-03 5.00000000e-02 3.91802709e-01
 8.08703949e+01 2.00000000e-01 8.00000000e-01 5.00000000e-01
 7.00000000e-01]


OverflowError: cannot fit 'int' into an index-sized integer

In [ ]:
print(res.x)